In [9]:
import cv2
import numpy as np
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
def readImage(img_path):
    img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
    return cv2.resize(img, (150, 150))

In [10]:
def getDescriptors(sift, img):
    location, des = sift.detectAndCompute(img, None)
    return location, des

In [11]:
def vstackDescriptors(descriptor_list):
    descriptors = np.array(descriptor_list[0])
    for descriptor in descriptor_list[1:]:
        descriptors = np.vstack((descriptors, descriptor)) 
    return descriptors

In [12]:
def clusterDescriptors(descriptors, no_clusters):
    kmeans = KMeans(n_clusters=no_clusters, random_state=0).fit(descriptors)
    return kmeans

In [13]:
def generate_bovw_spatial_histogram(im, location, kmeans, division):
    Y_div, X_div = division
    height, width = im.shape[:2]
    hist_bins = kmeans.n_clusters
    spatial_hist = np.zeros((Y_div * X_div, hist_bins))    
    div_height = height / Y_div
    div_width = width / X_div    
    for point, descriptor in zip(kp, kmeans.labels_):
        y, x = point.pt
        div_y = int(y // div_height)
        div_x = int(x // div_width)
        div_index = div_y * X_div + div_x
        spatial_hist[div_index, descriptor] += 1        
    return spatial_hist.flatten()

In [14]:
# Simulated process
image_path = 'shapes.png' 
image = readImage(image_path)
sift = cv2.SIFT_create()
# calling descriptors
location, descriptors = getDescriptors(sift, image)
# calling function for clustering
descriptors_flat = np.array(descriptors).reshape(-1, 128)
# Perform KMeans clustering
num_clusters = 10  
kmeans = clusterDescriptors(descriptors_flat, num_clusters)
# Generate spatial histograms for test cases
Test1 = [1, 1]
Test2 = [2, 2]
Test3 = [2, 3]
histogram_coarse = generate_bovw_spatial_histogram(image, location, kmeans, Test1)
histogram_medium = generate_bovw_spatial_histogram(image, location, kmeans, Test2)
histogram_fine = generate_bovw_spatial_histogram(image, location, kmeans, Test3)
print("For 1,1")
print(histogram_coarse)
print("For 2,2")
print(histogram_medium)
print("For 2,3")
print(histogram_fine)

C:\Users\kunal\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
C:\Users\kunal\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1440: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(


NameError: name 'kp' is not defined